# Reconocimiento de Patrones
[Curso de Reconocimiento de Patrones](https://https://domingomery.ing.puc.cl/teaching/patrones/)

Departamento de Ciencia de la Computación

Universidad Catolica de Chile

(c) Domingo Mery, http://domingomery.ing.uc.cl



# Ejercicio 06: Selección de Características para la Detección de espinas de salmón

En este ejercicio se realizará la clasificación entre dos clases de "parches" de radiogarfías de filetes de salmón de 100x100 pixeles (en escala de grises):

* Clase 0: Espinas (contiene parches de filetes de salmón con espinas). [Ver imágenes](https://www.dropbox.com/s/8bbb7yzyrjkl0qs/fishbones_0.png?dl=00)

* Clase 1: No-Espinas (contiene parches de filetes de salmón sin espinas) [Ver imágenes](https://www.dropbox.com/s/tdgp3shvk47213c/fishbones_1.png?dl=0)

La base de datos contiene 680 imágenes de 100x100 pixeles (340 imágenes por clase).

**ADVERTENCIA:** Este ejercicio tiene fines pedagógicos sólamente, con la idea de que la solución a este problema pueda ejecutarse en un par de minutos. Un buen detector de espinas sigue esta idea pero con miles de radiografías por clase.




# Setup inicial

Liberías necesarias para que funcione el algoritmo.

## Instalación de Balu3


In [ ]:
#
# EJECUTAR ESTA CELDA SIN MODIFICARLA
#

!git clone https://github.com/domingomery/balu3
!pip install ./balu3

fatal: destination path 'balu3' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./balu3
  Preparing metadata (setup.py) ... done
  Created wheel for balu3: filename=balu3-1.0-py3-none-any.whl size=43714 sha256=95c32d36e017efec3afceea99c92289dea91d4a777a3457103d55d8aae49ead5
  Stored in directory: /tmp/pip-ephem-wheel-cache-vjvao_0f/wheels/16/25/1e/631c042e986997d66481af1f42bd9a1f0937c379ca8d6f32fb
Successfully built balu3
  Attempting uninstall: balu3
    Found existing installation: balu3 1.0
    Uninstalling balu3-1.0:
      Successfully uninstalled balu3-1.0


## Setup de librerías

In [ ]:
#
# EJECUTAR ESTA CELDA SIN MODIFICARLA
#

import numpy as np
import matplotlib.pyplot as plt

from   tqdm.auto import tqdm

from   balu3.fx.chr    import lbp
from   balu3.fs.sel    import jfisher,sfs,exsearch
from   balu3.cl.basics import SplitTrainTest # Separación entre training y testing
from   balu3.cl.basics import ClassifierKNN  # Clasificación usando KNN
from   balu3.io.misc   import imageload
from   balu3.ft.norm   import minmax

print('Librerías cargadas.')

Librerías cargadas.


## Funciones necesarias

## ADVERTENCIA

<font color='red'> **PARA LA EJECUCIÓN DE ESTA TAREA NO DEBE USAR OTRAS LIBRERÍAS**

(sólo está permitido usar las librerías definidas en las celdas anteriores)

# PREGUNTA 1: Carga de base de datos

(1 punto)

La base de datos consiste en 2 clases con 340 imágenes por clase. Se almacenan en la carpeta `fishbones` con el formato `fish_xx_nnnn.png`, donde `xx` es el ID de la clase (00 para espinas, 02 para no-espinas) y `nnnn` es el número de la foto de la clase (0001, 0002, ... 0340). Las radiografías son de 100x100 pixeles en escala de grises.


En esta celda debes cargar la base de datos a este ambiente de trabajo, descargando el archivo fishbones.zip que se encuentra en este link: `https://www.dropbox.com/s/7d9y6kllguegk77/fishbones.zip`. Una vez descargado el archivo zip, debes descomprimirlo.

AYUDA: usar comando `!wget <url archivo zip>` para descargar el archivo, y el comando `!unzip <archivo zip>` para descomprimirlo.

<font color='red'> **SALIDA ESPERADA:** 

* carpeta `fishbones` creada con 680 imágenes `fish_xx_nnnn.png`




In [ ]:
 # INGRESE CODIGO DE PREGUNTA 1 AQUI
!wget https://www.dropbox.com/s/7d9y6kllguegk77/fishbones.zip
!unzip fishbones.zip
print("Done")

--2023-04-25 19:29:36--  https://www.dropbox.com/s/7d9y6kllguegk77/fishbones.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/7d9y6kllguegk77/fishbones.zip [following]
--2023-04-25 19:29:37--  https://www.dropbox.com/s/raw/7d9y6kllguegk77/fishbones.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc159459b5685a8276d8b1c5c1c0.dl.dropboxusercontent.com/cd/0/inline/B63n2vXTjHxmHy55cB34pTfsDiJBNP_ynQ1DTusmzxd-2l5OFg_yQd65IlZ0Gyy2rLqFGeBbd2o6asL0cqCl8rtr1-smoWw3bsrdRamC543PX8TlxkxadxpIRv1bXsLnc4u90WbgkPUIVxiNoXLKpZo-h6nJgQS_FlwREpG_l9g81w/file# [following]
--2023-04-25 19:29:37--  https://uc159459b5685a8276d8b1c5c1c0.dl.dropboxusercontent.com/cd/0/inline/B63n2vXTjHxmHy55cB34pTfsDiJBNP_ynQ1DTusmzxd-2l5OFg_yQd65IlZ0Gyy2rLqFGeBbd2o

# PREGUNTA 2: Extracción de características

(2 puntos)

En esta pregunta se debe realizar la extracción de características de todas las imágenes de la base de datos. Las características a extraer son las basadas en LBP (invariantes a la rotación y con norma unitaria) y se extraen usando el siguiente comando de la librería Balu3:


`f  = lbp(img, hdiv=a, vdiv=b, mapping='uniform', norm=True)`

En este caso una imagen `img` se divide en `a x b` particiones y en cada una de ellas se extrae el descriptor LBP (vector de 10 elementos) cuya norma es uno. En este caso `f` es un vector de `10*a*b` elementos. En este ejercicio trabajaremos con `a = b = 1`, es decir sólo una partición por imagen.




<font color='red'> **SALIDA ESPERADA:** 

* `Xlbp`: una matriz de 680 x 10 elementos. Las primeras 340 filas corresponden a los LBP de las imágenes de la clase 0, mientras que las siguientes 340 filas corresponden a los LBP de las imágenes de la clase 1.

* `y`: vector con el Ground Truth de cada muestra. <font color='red'>Este vector debe ser un array del tipo `int` con 680 elementos (340 ceros y 340 unos). 

<font color='orange'> Para escribir este código reutilice los códigos vistos en clase


In [ ]:
# INGRESE CODIGO DE PREGUNTA 2 AQUI
K = 2
N = 340

Mlbp = 10

Xlbp = X = np.zeros(( K * N, Mlbp ))
y0 = np.zeros(( N ), 'int')
y1 = np.ones(( N ), 'int')
y = np. concatenate((y0, y1), axis=0)

t = 0

a = b = 1

print('Cargando imagenes y extrayendo características...')
for j in range(K):            # para cada clase
  print('Extrayendo características: batch '+str(j+1)+'/'+str(K))
  for i in tqdm(range(N)):
    # Lectura de la imagen
    clase     = j
    num_img   = i+1
    img       = 255*imageload('fishbones/fish_',clase,2,num_img,4) 

    # LBP clásico de 1x1 particiones
    X[t,:] = lbp(img, hdiv=a, vdiv=b, mapping='uniform', norm=True)

    y[t] = j # la muestra t pertenece a la clase j
    t = t+1


Cargando imagenes y extrayendo características...
Extrayendo características: batch 1/2


  0%|          | 0/340 [00:00<?, ?it/s]

Extrayendo características: batch 2/2


  0%|          | 0/340 [00:00<?, ?it/s]

# PREGUNTA 3: Definición de Training y Testing

(1 punto)

<font color='red'> IMPORTANTE: 

En todos los experimentos usaremos las primeras 280 muestras por clase para training y el resto (60) para testing.

Para la separación train/test use la función `SplitTrainTest` cuya sintaxis es la siguiente:

`Xtrain,ytrain,Xtest,ytest = SplitTrainTest(X,y,n)` 

donde las primeras `n` muestras de cada clase son tomadas para el training y el resto para el testing. Esta función divide `X` en `Xtrain` y `Xtest`, y a la vez divide `y` en `ytrain` e `ytest`.


<font color='red'> **SALIDA ESPERADA:** 

Variables:

* `Xtrain,ytrain,Xtest,ytest`.






In [ ]:
# INGRESE CODIGO DE PREGUNTA 3 AQUI
n = 280
Xtrain,ytrain,Xtest,ytest = SplitTrainTest(X,y,n)

# PREGUNTA 4: Normalización de las características

(1 punto)

Usando la función `minmax` de Balu3 normalice los datos de training. El uso de esta función se encuentra en el ejemplo de Colab "Ejemplo de Seleccion de Caracteristicas" visto en la última clase.


<font color='red'> **SALIDA ESPERADA:** 

El output de esta celda debe ser:

* `Xtrain_norm` : Datos del training normalizados

* `Xtest_norm` : Datos del testing normalizados

<font color='orange'>Se recomienda estudiar bien las dos líneas de código que hacen la normalización.</font>





In [ ]:
# INGRESE CODIGO DE PREGUNTA 4 AQUI
Xtrain_norm, a, b = minmax(Xtrain)
Xtest_norm        = Xtest * a + b

# PREGUNTA 5: Clasificación sin Selección de Características

(1 punto)

<font color='red'>**IMPORTANTE**: En TODO este Ejercicio (E05), el clasificador será SIEMPRE un **KNN de 5 vecinos**. </font>


Usando **todos** los datos normalizados y la función `ClassifierKNN` con `n_neighbors = 5` obtenga el Accuracy la matriz de confusión. En esta celda debeemplear todas las características (sin realizar selección alguna).

<font color='red'> **SALIDA ESPERADA:** 

* `acc_all`: Accuracy obtenido

* `C_all`: Matriz de confusión

<font color='red'>Para definir el número de vecinos del KNN en la función `ClassifierKNN` incluya el parámetro `n_neighbors = 5`.</font>





In [ ]:
# INGRESE CODIGO DE PREGUNTA 5 AQUI
acc_all, C_all = ClassifierKNN(Xtrain_norm, ytrain, Xtest_norm, ytest, n_neighbors=5)

Entrenando con 560 muestras y probando con 120 muestras (de 10 características)...
Testing Accuracy = 79.16666666666666%
Matriz de Confusión:
[[46 14]
 [11 49]]


# PREGUNTA 6: Selección de características usando "Búsqueda Exhaustiva"

(8 puntos)

En esta pregunta buscaremos las q=4 mejores características de las m=10 extraídas. La búsqueda exhaustiva -en este caso- evalúa todas las combinaciones de 4 características (sin repetición), es decir, evalúa las siguientes m!/(q! x (m-q!) = 10!/(4! x (10-4)!) = 210 combinaciones:

* Combinación 1: (0,1,2,<font color='orange'> 3 </font> )

* Combinación 2: (0,1,2,<font color='orange'> 4 </font>)

* Combinación 3: (0,1,2,<font color='orange'> 5 </font>)

* Combinación 4: (0,1,2,<font color='orange'> 6 </font>)

* Combinación 5: (0,1,2,<font color='orange'> 7 </font>)

* Combinación 6: (0,1,2,<font color='orange'> 8 </font>)

* Combinación 7: (0,1,2,<font color='orange'> 9 </font>)

* Combinación 8: (0,1,<font color='magenta'> 3 </font>,<font color='cyan'> 4 </font>) -> la combinación (0,1,3,0) no se evalúa porque se repite el 0, además la combinación (0,1,3,2) tampoco se evalúa porque ya fue evaluada en la combinación 1 (en otro orden).

* Combinación 9: (0,1,<font color='magenta'> 3 </font>,<font color='cyan'> 5 </font>)

* Combinación 10: (0,1,<font color='magenta'> 3 </font>,<font color='cyan'> 6 </font>)

* :

* Combinación 210: (6,7,8,9)

La búsqueda exhaustiva escoge la "mejor" de estas 210 combinaciones. La mejor combinación será aquella que tenga la mayor "separabilidad" entre las clases. Para obtener un "score" de la separabilidad de las características podemos usar la función de Fisher que mide qué tan compactas están cada una de las clases, y qué tan separadas están entre ellas. Una buena separabilidad (un buen "score") se obtiene si en el espacio de características, las clases son compactas y la separación entre ellas es grande. Este "score" está implementado en la función `jfisher`. Entre mayor sea el "score", entonces mayor será la separabilidad. En la búsqueda exhaustiva buscamos aquella combinación que maximiza el score entregado por `jfisher`.

## Ejemplo de cómo usar `jfisher`:

A manera de ejemplo, vamos a suponer que hemos almacenado en la matriz `Z` la extracción de 20 características de 1000 muestras, es decir el tamaño de esta matriz `Z` es de 1000x20. Adicionalmente, la clasificación ideal de estas 1000 muestras se encuentra almacenada en la variable `d` de 1000 elementos. En este ejemplo, los elementos de `d` toman los valores 0, 1, 2 ó 3 (porque hay 4 clases).

En este ejemplo, deseamos evaluar el "score" de las características (3,15,17). Para evaluar el desempeño de estas tres características seguimos estos cuatro pasos:

* Definimos `i1=3`, `i2=15`, y `i3=17`.

* Definimos `ii = (i1,i2,i3)`.

* Escogemos de `Z` las columnas indexadas por `ii` usando `Z_sel = Z[:,ii]`. Ahora `Z_sel` tiene 1000 muestras y 3 columnas. 

* Para obtener el "score" de estas características seleccionadas, usamos el comando: `sc = jfisher(Z_sel,d)`

En este ejemplo, en la variable `sc` hemos almacenado el "score", es decir la evaluación de la función de Fisher de la separabilidad para la selección de las columas (3,15,17).

En el siguiente código Ud. debe implementar la búsqueda exhaustiva. El input de este código es la matriz `Xtrain` de 640x10 elementos, y el vector `ytrain` de 640 elementos, la salida debe ser el vector `ex_indices` de 4 números distintos entre 0 y 9 (ya que existen 10 características en `Xtrain` y se escogen 4), que corresponde a la mejor combinación de características. 

En este código Ud. debe hacer  210 evaluaciones de la función de Fisher, una para cada combinación, y se selecciona aquella combinación que maximiza la función de Fisher.

La búsqueda exhaustiva se realiza con cuatro contadores `(i1,i2,i3,i4)` para cada una de las 4 características. Como nuestros datos tienen 10 características, es necesario observar que 

* `i1` debe contar de 0 a 6

* `i2` debe contar de `i1+1` a 7

* `i3` debe contar de `i2+1` a 8

* `i4` debe contar de `i3+1` a 9


En esta implemantación, `i1 < i2 < i3 < i4 `, de esta manera 1) se recorren todos los índices sin repetirse, es decir la combinación (1,1,2,3) no debe evaluarse, y 2) no se evalúan combinaciones que ya han sido evaluadas anteriormente, es decir si evaluó (1,2,3,4), no se evalúa (3,2,1,4) ya que ambas combinaciones tienen el mismo "score".

<font color='red'> **SALIDA ESPERADA:** 

Vector `ex_indices` con 4 elementos.

In [ ]:
#######################################################################
# AYUDA: Hay que leer y entender el texto anterior (de la Pregunta 6) #
#######################################################################

m = Xtrain_norm.shape[1]                            # <= COMPLETAR AQUI EL NÚMERO DE CARACTERÍSTICAS DE Xtrain_norm, NO PONER m = 10 
score_max = 0                    # Inicialización del score máximo
ex_indices = (-1,-1,-1,-1)       # Inicialización de las columnas seleccionadas
p = 0                            # Contador de evaluaciones
for i1 in range(7):            # <= COMPLETAR AQUI EL RANGO DE LA VARIABLE i1 (PRIMERA COLUMNA SELECCIONADA)
  for i2 in range(i1+1, 8):          # <= COMPLETAR AQUI EL RANGO DE LA VARIABLE i2 (SEGUNDA COLUMNA SELECCIONADA)
    for i3 in range(i2+1, 9):        # <= COMPLETAR AQUI EL RANGO DE LA VARIABLE i3 (TERCERA COLUMNA SELECCIONADA)
      for i4 in range(i3+1, 10):      # <= COMPLETAR AQUI EL RANGO DE LA VARIABLE i4 (CUARTA COLUMNA SELECCIONADA)
            p = p+1
            ii = (i1,i2,i3,i4)
            Xtrain_sel = Xtrain_norm[:,ii]     # <= COMPLETAR AQUI CON LAS 4 COLUMNAS DE Xtrain_norm INDICADAS POR ii
            score = jfisher(Xtrain_sel, ytrain)          # <= COMPLETAR AQUI CON LA FUNCIÓN DE FISHER PARA OBTENER EL SCORE DE Xtrain_sel
            if score > score_max:
              score_max = score
              ex_indices = ii
              print('nuevo máximo encontrado: score = '+"{:.4f}".format(score_max)+' para la combinación '+str(ii))
print('Se evaluaron '+str(p)+' combinaciones.')
print('Las 4 características seleccionadas son: '+str(ex_indices))


nuevo máximo encontrado: score = 0.4642 para la combinación (0, 1, 2, 3)
nuevo máximo encontrado: score = 1.3319 para la combinación (0, 1, 2, 4)
nuevo máximo encontrado: score = 1.6411 para la combinación (0, 1, 4, 8)
nuevo máximo encontrado: score = 1.7471 para la combinación (0, 3, 4, 8)
Se evaluaron 210 combinaciones.
Las 4 características seleccionadas son: (0, 3, 4, 8)


# PREGUNTA 7: Clasificación con características seleccionadas por búsqueda exhaustiva

(3 puntos)

A continuación se definen las matrices `Xtrain_ex` (de 640x4 elementos) y `Xtest_ex` (de 40x4) elementos, tomando las columnas seleccionadas de `Xtrain_norm` y `Xtest_norm` respectivamente usando los índices encontrados en `ex_indices`.

Finalmente, entrenamos y probamos el clasificador con estos nuevos datos (no olvidar que el KNN es con 5 vecinos). No es necesario definir `ytrain_ex` y `ytest_ex` porque son iguales a `ytrain` y `ytest` respectivamente.

<font color='red'> **SALIDA ESPERADA:** 

* `Xtrain_ex`: matriz de características de training que han sido seleccionadas de `Xtrain_norm`

* `Xtest_ex`: matriz de características de testing que han sido seleccionadas de `Xtest_norm`

* `acc_ex`: Accuracy obtenido

* `C_ex`: Matriz de confusión


In [ ]:
# INGRESE CODIGO DE PREGUNTA 7 AQUI
Xtrain_ex = Xtrain_norm[:,ex_indices]
Xtest_ex  = Xtest_norm[:,ex_indices] 
# print(Xtrain_ex.shape)
# print(Xtest_ex.shape)
acc_ex, C_ex = ClassifierKNN(Xtrain_ex, ytrain, Xtest_ex, ytest)

Entrenando con 560 muestras y probando con 120 muestras (de 4 características)...
Testing Accuracy = 92.5%
Matriz de Confusión:
[[58  2]
 [ 7 53]]


# PREGUNTA 8: Selección con Búsqueda Exhaustiva usando Balu3

(1 punto)

Empleando la función `exsearch` de Balu3, obtenga el mismo resultado de la Pregunta 6. La función `exsearch` tiene la siguiente sintaxis:

`ss = exsearch(Z,d,p)`

donde 

* `ss` es el vector de índices seleccionados,

* `Z` es la matriz original de características, 

* `d` es el vector de la clasificación ideal, y 

* `p` es el número de características a seleccionar.

<font color='red'> **SALIDA ESPERADA:** 

El output de esta celda debe ser el vector `ex2_indices` de 4 elementos que debe contener los mismos elementos que `ex_indices` obtenidos en la Pregunta 6. Imprima con print el resultado.


In [ ]:
# INGRESE CODIGO DE PREGUNTA 8 AQUI
ex2_indices = exsearch(Xtrain, ytrain, len(ex_indices))
# ex2_indices = exsearch(X, y, len(ex_indices))
print(ex2_indices)
print(ex_indices)

[0 3 4 8]
(0, 3, 4, 8)


# PREGUNTA 9: Selección con SFS usando Balu3

(1 punto)

Empleando la función `sfs` de Balu3 seleccione 4 características, y realice la clasificación con las características seleccionadas. 

La sintaxis de `sfs` es similar a la de `exsearch`, es decir:

`ss = sfs(Z,d,p)`

donde 

* `ss` es el vector de índices seleccionados,

* `Z` es la matriz original de características, 

* `d` es el vector de la clasificación ideal, y 

* `p` es el número de características a seleccionar.

<font color='red'> **SALIDA ESPERADA:** 

El output de esta celda debe ser el vector `sfs_indices` de 4 elementos. Imprima el vector usando `print`.


In [ ]:
# INGRESE CODIGO DE PREGUNTA 9 AQUI
sfs_indices = sfs(X, y, 4)
print(sfs_indices)

[4 8 1 3]


# PREGUNTA 10: Clasificación usando las características seleccionadas por SFS

(2 puntos)

A continuación se definen las matrices `Xtrain_sfs` (de 640x4 elementos) y `Xtest_sfs` (de 40x4) elementos, tomando las columnas seleccionadas de `Xtrain_norm` y `Xtest_norm` respectivamente usando los índices encontrados en `sfs_indices`.

No es necesario definir `ytrain_sfs` y `ytest_sfs` porque son iguales a `ytrain` y `ytest` respectivamente.

Finalmente, entrenamos y probamos el clasificador con estos nuevos datos. No olvidar que el clasificador es un KNN con 5 vecinos. 

<font color='red'> **SALIDA ESPERADA:** 

* `Xtrain_sfs`: matriz de características de training que han sido seleccionadas de `Xtrain_norm`

* `Xtest_sfs`: matriz de características de testing que han sido seleccionadas de `Xtest_norm`

* `acc_sfs`: Accuracy obtenido

* `C_sfs`: Matriz de confusión


In [ ]:
# INGRESE CODIGO DE PREGUNTA 10 AQUI
Xtrain_sfs, Xtest_sfs =  Xtrain_norm[:,sfs_indices], Xtest_norm[:,sfs_indices]
acc_sfs, C_sfs = ClassifierKNN(Xtrain_sfs, ytrain, Xtest_sfs, ytest, n_neighbors=5)

Entrenando con 560 muestras y probando con 120 muestras (de 4 características)...
Testing Accuracy = 77.5%
Matriz de Confusión:
[[45 15]
 [12 48]]


In [ ]:
#
# EJECUTAR ESTA CELDA SIN MODIFICARLA
#

print('P02:        Xlb(12,1):',Xlbp[12,1])
print('P02:  GT y(10),y(600):',y[10],y[600])
print('P03:      Xtrain(5,2):',Xtrain[5,2])
print('P03:       Xtest(7,3):',Xtest[7,3])
print('P04: Xtrain_norm(5,2):',Xtrain_norm[5,2])
print('P04:  Xtest_norm(7,3):',Xtest_norm[7,3])
print('P05:          acc_all:',acc_all*100)
print('P05:            C_all:',C_all[0,:],C_all[1,:])
print('P06:       ex_indices:',ex_indices)
print('P04:   Xtrain_ex(5,2):',Xtrain_ex[5,2])
print('P04:    Xtest_ex(7,3):',Xtest_ex[7,3])
print('P07:           acc_ex:',acc_ex*100)
print('P07:             C_ex:',C_ex[0,:],C_ex[1,:])
print('P08:      ex2_indices:',ex2_indices)
print('P09:      sfs_indices:',sfs_indices)
print('P10:  Xtrain_sfs(5,2):',Xtrain_sfs[5,2])
print('P10:   Xtest_sfs(7,3):',Xtest_sfs[7,3])
print('P10:          acc_sfs:',acc_sfs*100)
print('P10:            C_sfs:',C_sfs[0,:],C_sfs[1,:])


P02:        Xlb(12,1): 0.15630490026453256
P02:  GT y(10),y(600): 0 1
P03:      Xtrain(5,2): 0.10637916219523334
P03:       Xtest(7,3): 0.259003473201129
P04: Xtrain_norm(5,2): 0.478981177000125
P04:  Xtest_norm(7,3): 0.5189001306430938
P05:          acc_all: 79.16666666666666
P05:            C_all: [46 14] [11 49]
P06:       ex_indices: (0, 3, 4, 8)
P04:   Xtrain_ex(5,2): 0.5213551222563919
P04:    Xtest_ex(7,3): 0.36359826574137083
P07:           acc_ex: 92.5
P07:             C_ex: [58  2] [ 7 53]
P08:      ex2_indices: [0 3 4 8]
P09:      sfs_indices: [4 8 1 3]
P10:  Xtrain_sfs(5,2): 0.6145039105766853
P10:   Xtest_sfs(7,3): 0.5189001306430938
P10:          acc_sfs: 77.5
P10:            C_sfs: [45 15] [12 48]


# VERIFICACION FINAL

<font color='red'> **ADVERTENCIA:** Este ejercicio será evaluado de la siguiente manera: si el código funciona y el resultado es correcto, la pregunta tendrá un 100% de la evaluación, en caso contrario 0%. Para asegurarse que el código se ejecute sin caídas seleccione la opción del menú:

1) <font color='orange'> 'Runtime' + 'Disconnect and delete runtime' (inicializa todo)

2) <font color='orange'> 'Run all'

<font color='red'> El código debería ejecutarse desde el inicio hasta el final sin problema alguno.

# PASO FINAL: Subir tarea

<font color='orange'> **ADVERTENCIA FINAL:** Este archivo, que debe llamarse [tu RUT].ipynb, (el RUT debe ser sin puntos ni guion) y debe subirse a Canvas según la hora señalada para la actividad E06.